## Natural Language Processing

### About Dataset

#### Context
The SMS Spam Collection is a set of SMS tagged messages that have been collected for SMS Spam research. It contains one set of SMS messages in English of 5,574 messages, tagged acording being ham (legitimate) or spam.

#### Content
The files contain one message per line. Each line is composed by two columns: v1 contains the label (ham or spam) and v2 contains the raw text.

This corpus has been collected from free or free for research sources at the Internet:

-> A collection of 425 SMS spam messages was manually extracted from the Grumbletext Web site. This is a UK forum in which cell phone users make public claims about SMS spam messages, most of them without reporting the very spam message received. The identification of the text of spam messages in the claims is a very hard and time-consuming task, and it involved carefully scanning hundreds of web pages. The Grumbletext Web site is: [Web Link](http://www.grumbletext.co.uk/).

-> A subset of 3,375 SMS randomly chosen ham messages of the NUS SMS Corpus (NSC), which is a dataset of about 10,000 legitimate messages collected for research at the Department of Computer Science at the National University of Singapore. The messages largely originate from Singaporeans and mostly from students attending the University. These messages were collected from volunteers who were made aware that their contributions were going to be made publicly available. The NUS SMS Corpus is avalaible at: [Web Link](http://www.comp.nus.edu.sg/~rpnlpir/downloads/corpora/smsCorpus/).

-> A list of 450 SMS ham messages collected from Caroline Tag's PhD Thesis available at [Web Link](http://etheses.bham.ac.uk/253/1/Tagg09PhD.pdf).

-> Finally, we have incorporated the SMS Spam Corpus v.0.1 Big. It has 1,002 SMS ham messages and 322 spam messages and it is public available at: [Web Link](http://www.esp.uem.es/jmgomez/smsspamcorpus/). This corpus has been used in the following academic researches:

#### Acknowledgements
The original dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection). The creators would like to note that in case you find the dataset useful, please make a reference to previous paper and the web page: http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/ in your papers, research, etc.

We offer a comprehensive study of this corpus in the following paper. This work presents a number of statistics, studies and baseline results for several machine learning methods.

Almeida, T.A., GÃ³mez Hidalgo, J.M., Yamakami, A. Contributions to the Study of SMS Spam Filtering: New Collection and Results. Proceedings of the 2011 ACM Symposium on Document Engineering (DOCENG'11), Mountain View, CA, USA, 2011.

Inspiration
Can you use this dataset to build a prediction model that will accurately classify which texts are spam?


[Reference](https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset)


In [1]:
import random
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [2]:
from nltk import wordnet

In [3]:
from nltk import corpus

### Importing Data

In [4]:
spam = pd.read_csv("../Data/SMSSpamCollections.csv", sep = "\t",names=["Label", "Message"])

In [5]:
print(spam.head())

                                               Label  Message
0  ham,"Go until jurong point, crazy.. Available ...      NaN
1                  ham,Ok lar... Joking wif u oni...      NaN
2  spam,Free entry in 2 a wkly comp to win FA Cup...      NaN
3  ham,U dun say so early hor... U c already then...      NaN
4  ham,"Nah I don't think he goes to usf, he live...      NaN


### Converting the DataFrame into List of tuples: Each tuple contains a labal and a message

In [6]:
data_set = []
for index, row in spam.iterrows():
    data_set.append((row['Message'], row['Label']))

In [7]:
print(data_set[:5])

[(nan, 'ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."'), (nan, 'ham,Ok lar... Joking wif u oni...'), (nan, "spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"), (nan, 'ham,U dun say so early hor... U c already then say...'), (nan, 'ham,"Nah I don\'t think he goes to usf, he lives around here though"')]


In [8]:
print(len(data_set))

5574


### Preprocessing

In [9]:
stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

##### Changes document to lower case, remove stopwords and lemmatizes/stems:

In [10]:
def preprocess(document, stem=True):
    document = document.lower()
    words = word_tokenize(document)
    words = [word for word in words if word not in stopwords.words("english")]
    if stem:
        words = [stemmer.stem(word) for word in words]
    else:
        words = [wordnet_lemmatizer.lemmatize(word, pos='v') for word in words]
        
    document = " ".join(words)
    return document

##### Preprocessing on data_set:

In [11]:
messages_set = []
for (message, label) in data_set:
    words_filtered = [e.lower() for e in preprocess(message, stem=False).split() if len(e) >=3]
    messages_set.append((words_filtered, label))

AttributeError: 'float' object has no attribute 'lower'

In [13]:
print(messages_set[:5])

[(['jurong', 'point', 'crazy', 'available', 'bugis', 'great', 'world', 'buffet', '...', 'cine', 'get', 'amore', 'wat', '...'], 'ham'), (['lar', '...', 'joke', 'wif', 'oni', '...'], 'ham'), (['free', 'entry', 'wkly', 'comp', 'win', 'cup', 'final', 'tkts', '21st', 'may', '2005.', 'text', '87121', 'receive', 'entry', 'question', 'std', 'txt', 'rate', 'apply', '08452810075over18'], 'spam'), (['dun', 'say', 'early', 'hor', '...', 'already', 'say', '...'], 'ham'), (['nah', "n't", 'think', 'usf', 'live', 'around', 'though'], 'ham')]


### Creating Features

##### Single list of words in the entire dataset:

In [14]:
def get_words_in_messages(messages):
    all_words = []
    for (message, label) in messages:
        all_words.extend(message)
    return all_words

##### Final feature list using intuitive FreqDist, to eliminate all the duplicate words:

In [15]:
def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

In [16]:
word_features = get_word_features(get_words_in_messages(messages_set))
print(len(word_features))

8003


### Creating a Train and Test Datasets

##### Creating slicing index at 80% threshold:

In [17]:
sliceIndex = int((len(messages_set) * .8))

##### Shuffling the pack to create a random and unbiased split of the dataset:

In [18]:
train_messages, test_messages = messages_set[:sliceIndex], messages_set[sliceIndex:]

In [19]:
len(train_messages)
len(test_messages)

1115

### Creating Feature Maps for Train and Test Data

In [20]:
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%5)' % word] = (word in document_words)
    return features

In [21]:
training_set = nltk.classify.apply_features(extract_features, train_messages)
testing_set = nltk.classify.apply_features(extract_features, test_messages)

In [22]:
print('Training set size: ', len(training_set))
print('Test set size: ', len(testing_set))

Training set size:  4457
Test set size:  1115


### Training

In [23]:
classifier = nltk.NaiveBayesClassifier.train[training_set]

TypeError: 'method' object is not subscriptable